# Incremental Forecast generation with MLForecast

This article presents a practical framework for implementing incremental forecasting to build scalable, production-grade time series pipelines using MLForecast.

## Introduction

When a forecasting system goes live, the flow of data doesn’t stop. Each month - or even each week - new observations arrive, and decision-makers expect your forecasts to adjust in real time. Yet, retraining the entire model every time new data comes in is often the default reaction. It seems simple, but it’s computationally expensive, time-consuming, and can introduce instability into production workflows.

This is where incremental forecasting becomes essential. Instead of retraining from scratch, we can incrementally update the model to reflect the latest data while preserving its learned patterns and parameters. In Nixtla’s `MLForecast`, the `update()` method provides this capability by allowing a trained forecasting object to absorb new observations without retraining the underlying estimator. It extends the historical window for each series, recalculates lag and date features, and ensures that subsequent forecasts are generated using the most recent actuals.